In [43]:
def simulate_city(n_sectors, n_tech, tech_correlation, sector_weights, endowment_draws=None, shape=2.0, seed=123, random=0):
    import numpy as np 
    from scipy.stats import weibull_min
    import random

    # Set the seed if not random
    if random == 0:
        np.random.seed(seed)
    elif random == 1:
        np.random.seed()

    # Check if the number of technologies matches the number of correlations
    if len(tech_correlation) != n_tech:
        raise ValueError("The number of technology correlations must match the number of technologies")

    # Generate endowment draws if not provided
    if endowment_draws is None:
        endowment_draws = np.random.normal(loc=0, scale=1, size=n_tech)
        endowment_draws = np.array([np.absolute(endowment_draws)])
    elif len(endowment_draws) != n_tech:
        raise ValueError("The number of endowment draws must match the number of technologies")

    # Generate sector specific weight draws
    if len(sector_weights) != n_sectors:
        raise ValueError("The number of sector weights must match the number of sectors")
    sector_weights = sector_weights * 10

    # Calculate powered weighted endowments
    powered_weights = np.power(sector_weights, tech_correlation)

    # Calculate weighted endowments
    weighted_endowments = endowment_draws * powered_weights

    # Generate productivity draws
    productivity_draws = weibull_min.rvs(shape, scale=weighted_endowments)

    # Assign an ID
    hex_value = hex(random.randint(0, 16**6))[2:]

    return {
        "id": hex_value,
        "endowment_draws": endowment_draws,
        "sector_weights": sector_weights,
        "tech_correlation": tech_correlation,
        "productivity_draws": productivity_draws
    }

def solve_wages(city, person):
    import numpy as np

    if city['productivity_draws'].shape != person['productivity_draws'].shape:
        raise ValueError("The shape of the city and person productivity draws must match")

    wages = city['productivity_draws'] * person['productivity_draws']
    wages = np.sum(wages, axis=1)

    max_indus = np.argmax(wages)
    max_wage = wages[max_indus]

    return {
        "person_id" : person['id'],
        "city_id" : city['id'],
        "max_wage": max_wage,
        "max_indus": max_indus,
        "wages": wages
    }



In [44]:
import numpy as np
from collections import Counter

np.random.seed(123)
n_sectors = 10
n_tech = 3
sector_weights = np.random.dirichlet(np.ones(n_tech), size=n_sectors)

# Simulate a city with 10 sectors and 3 technologies
city = simulate_city(10, 3, [0.7, 0.1, 0.5], sector_weights)
person = simulate_city(10, 3, [0.7, 0.1, 0.5], sector_weights)

wages = city['productivity_draws'] * person['productivity_draws']
wages = np.sum(wages, axis=1)

max_pos = np.argmax(wages)
max_wage = wages[max_pos]

print(f"The maximum value is {max_wage} and it's at position {max_pos}")

# Solve for the wages
result = solve_wages(city, person)

print(result['max_indus'])

The maximum value is 6.067326753997397 and it's at position 1
1


In [45]:
# Simulate one city and 500 people to demonstrate sorting of people into sectors 

np.random.seed(123)
n_sectors = 10
n_tech = 3
tech_correlation = np.array([0.7, 0.5, 0.3])
sector_weights = np.random.dirichlet(np.ones(n_tech), size=n_sectors)

n_people = 500

city = simulate_city(n_sectors, n_tech, tech_correlation, sector_weights, [0.6, 0.2, 0.3])

people = []
wages = []

for i in range (n_people):
    person = simulate_city(n_sectors, n_tech, tech_correlation, sector_weights, random=1)
    result = solve_wages(city, person)
    people.append(person)
    wages.append(result)

    person['max_indus'] = result['max_indus']
    person['max_wage'] = result['max_wage']
    person['city_id'] = result['city_id']

max_indus_counts = Counter([wage['max_indus'] for wage in wages])
print(max_indus_counts)

print(city['sector_weights'])
print(city['id'])

Counter({7: 172, 2: 152, 5: 73, 8: 40, 4: 31, 9: 16, 6: 9, 1: 7})
[[6.67332544 1.8866171  1.44005746]
 [3.05587213 4.8466022  2.09752568]
 [6.85774309 2.00411947 1.13813744]
 [2.23825567 1.89008262 5.87166171]
 [5.03562542 0.53675916 4.42761542]
 [7.72515708 1.16214922 1.1126937 ]
 [3.00510875 3.00744604 3.98744521]
 [4.58815448 3.12366462 2.28818089]
 [6.0435048  1.83900445 2.11749074]
 [1.61567108 2.16827491 6.21605401]]
6a6cbd


In [51]:
# Simulate productivity shock for first tech sector
# This should be done for each city, but demonstration is done on each person instead to show how the loop would work 

print(city['productivity_draws'])

value_to_add = 0.5 

new_wages = []

for person in people:
    person['productivity_draws'] = person['productivity_draws'] + value_to_add
    result = solve_wages(city, person)
    new_wages.append(result)

    new_wages['welfare_change'] = result['max_wage'] - person['max_wage']

    person['max_indus'] = result['max_indus']
    person['max_wage'] = result['max_wage']
    person['city_id'] = result['city_id']

max_indus_counts = Counter([wage['max_indus'] for wage in new_wages])
print(max_indus_counts)

[[0.7042696  0.20715149 0.25126597]
 [1.08190896 0.32796595 0.22922931]
 [1.72152441 0.42362068 0.52975191]
 [0.88034036 0.27192487 0.17884077]
 [1.14931079 0.10726052 0.66498297]
 [1.34772324 0.17512981 0.63767738]
 [1.10962155 0.33788998 0.16287798]
 [2.30620995 0.33977486 0.34128974]
 [1.36939609 0.16331378 0.27599815]
 [0.89767229 0.42505246 0.43862089]]


TypeError: list indices must be integers or slices, not str